# Overview

## Dataset

A public dataset from kaggle was used with minor altering in the naming patterns. The original can be found [here](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia)

The dataset is split into 3 folders:
- `test`
- `train`
- `val`

Each folder is futher split into `normal` and `pneumonia`. There a total of 5863 files stored as JPEGs.

# Setup

Import all required packages and set the base path for the datasets

In [70]:
import numpy as np
import pandas as pd
import os
import cv2
import skimage
import seaborn as sns

datasetsPath = "./datasets"

print("Path to dataset files:", datasetsPath)

labels = ['pneumonia', 'normal']
image_size = 150

Path to dataset files: ./datasets


In [71]:
def get_data (dir_relative_path):
    data = []
    for label in labels:
        currentPath = os.path.join(datasetsPath, dir_relative_path, label)
        print("Working inside folder: ", currentPath)
        class_number = labels.index(label)

        for image in os.listdir(currentPath):
            try:
                if image.endswith('.DS_Store'):
                    continue

                image_arr = cv2.imread(os.path.join(currentPath, image), cv2.IMREAD_GRAYSCALE)
                
                #print(os.path.join(currentPath, image))
                
                resized_arr = cv2.resize(image_arr, (image_size, image_size))
                
                data.append([resized_arr, class_number])
            except Exception as ex:
                print(ex)

    return np.array(data, dtype="object")

In [72]:
train = get_data('train/')
test = get_data('test/')
val = get_data("val")

Working inside folder:  ./datasets\train/pneumonia
Working inside folder:  ./datasets\train/normal
Working inside folder:  ./datasets\test/pneumonia
Working inside folder:  ./datasets\test/normal
Working inside folder:  ./datasets\val\pneumonia
Working inside folder:  ./datasets\val\normal


# Dat Augmentation
Import the necessary packages and resize the images

In [73]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,Dropout
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [74]:
train_datagen = ImageDataGenerator(rescale = 1. / 255, 
                                    horizontal_flip=0.4,
                                    vertical_flip=0.4,
                                    rotation_range=40, # randomly rotate the image up to 40 degrees TBD: such high value might be redundant
                                    shear_range=0.1, # skew the sides of the images by up to 10 degrees 
                                    width_shift_range=0.4, # shift the images left/right
                                    height_shift_range=0.3, # shift the images up/down TBD: needs more testing of values
                                    fill_mode="nearest")

valid_datagen = ImageDataGenerator(rescale= 1. / 255)
test_datagen = ImageDataGenerator(rescale= 1. / 255)



In [75]:
train_generator = train_datagen.flow_from_directory("./datasets/train",
                                  batch_size = 32,
                                  target_size=(image_size, image_size),
                                  class_mode = "categorical",
                                  shuffle=True,
                                  seed = 42,
                                  color_mode= "rgb")

test_generator = test_datagen.flow_from_directory("./datasets/test",
                                  batch_size = 32,
                                  target_size=(image_size, image_size),
                                  class_mode = "categorical",
                                  shuffle=True,
                                  seed = 42,
                                  color_mode= "rgb")

valid_generator = valid_datagen.flow_from_directory("./datasets/val",
                                  batch_size = 32,
                                  target_size=(image_size, image_size),
                                  class_mode = "categorical",
                                  shuffle=True,
                                  seed = 42,
                                  color_mode= "rgb")

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [76]:
class_labels = train_generator.class_indices

print(class_labels)

# Flip them around so we have "index : value"
class_name = {}

for value, index in class_labels.items():
    class_name[index] = value

print(class_name)

{'normal': 0, 'pneumonia': 1}
{0: 'normal', 1: 'pneumonia'}


In [77]:
base_model = VGG19(input_shape= (image_size, image_size, 3),
      include_top = False,
      weights = "imagenet")

for layer in base_model.layers:
    layer.trainable = False

flat = Flatten()(base_model.output)

# TODO: test and change around the neuron counts

class_1 = Dense(4608, activation= "relu")(flat)
dropout = Dropout(0.2)(class_1)

class_2 = Dense(1152, activation = 'relu')(dropout)
output_layer = Dense(2, activation = 'softmax')(class_2)

model_01 = Model(base_model.inputs, output_layer)
model_01.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 4608)           │    37,753,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4608)           │             

 Total params: 63,089,602 (240.67 MB)

 Trainable params: 43,065,218 (164.28 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [78]:
checkpointPath = "./modelCheckpoints/model.keras"

early_stop = EarlyStopping(monitor= "val_loss",
                   verbose= 1,
                   mode= "min",
                   patience= 4) # TODO: play around with patience

checkpoint = ModelCheckpoint(checkpointPath,
                monitor= "val_loss",
                save_best_only=True,
                save_weights_only=False,
                save_freq= "epoch",
                mode= "auto")

# TODO: play around with the values
rate_reduction = ReduceLROnPlateau(monitor="val_accuracy",
                        patience= 3, # Stop after X epochs of no improvement & reduce learning rate
                        verbose= 1,
                        factor= 0.5, # "learning rate * factor" after patience runs out (? correct word ?)
                        min_lr= 0.0001) # minimum learning rate

# Stochatic Gradient Descent
sgd = SGD(learning_rate= 0.0001, # Low initial value = more gradual updates
        momentum= 0.01, # TODO: play around with the momentum
        nesterov=True)


model_01.compile(loss="categorical_crossentropy", optimizer= sgd, metrics=["accuracy"])


In [79]:
history_01 = model_01.fit(train_generator, 
            steps_per_epoch=50,
            epochs=1, 
            callbacks=[early_stop, checkpoint, rate_reduction],
            validation_data=valid_generator)

C:\Users\kille\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Users\kille\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_152']
Received: inputs=Tensor(shape=(None, 150, 150, 3))
  warnings.warn(msg)


50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 654ms/step - accuracy: 0.5270 - loss: 0.7708 - val_accuracy: 0.5000 - val_loss: 0.8304 - learning_rate: 1.0000e-04


In [80]:
if not os.path.isdir("modelWeights/"):
    os.mkdir("modelWeights/")

model_01.save(filepath = "modelWeights/vgg19_model_01.keras", overwrite=True)

In [81]:
model_01.load_weights("modelWeights/vgg19_model_01.keras")

vgg_val_eval_01 = model_01.evaluate(valid_generator)
vgg_test_eval_01 = model_01.evaluate(test_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - accuracy: 0.5000 - loss: 0.8304
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 527ms/step - accuracy: 0.6308 - loss: 0.6834


In [82]:
print(f"Validation Loss: {vgg_val_eval_01[0]}")
print(f"Validation Accuarcy: {vgg_val_eval_01[1]}")
print(f"Test Loss: {vgg_test_eval_01[0]}")
print(f"Test Accuarcy: {vgg_test_eval_01[1]}")

Validation Loss: 0.8304159641265869
Validation Accuarcy: 0.5
Test Loss: 0.6809343099594116
Test Accuarcy: 0.625


In [83]:
# TODO: research if It's better to copy-paste the previous or just use it like this
# Copy-pasting should allow for changing of variables
model_02 = Model(base_model.inputs, output_layer)

model_02.load_weights("modelWeights/vgg19_model_01.keras")

set_trainable = False
for layer in base_model.layers:
    if layer.name in [ 'block5_conv3','block5_conv4']:
        set_trainable=True
    else:
        set_trainable = False
    layer.trainable = set_trainable
print(model_02.summary())

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 4608)           │    37,753,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4608)           │             

 Total params: 63,089,602 (240.67 MB)

 Trainable params: 47,784,834 (182.28 MB)

 Non-trainable params: 15,304,768 (58.38 MB)

None


In [84]:
# Stochatic Gradient Descent
sgd = SGD(learning_rate= 0.0001, # Low initial value = more gradual updates
        momentum= 0.01, # TODO: play around with the momentum
        nesterov=True)

model_02.compile(loss="categorical_crossentropy", optimizer= sgd, metrics=["accuracy"])

In [85]:
history_02 = model_02.fit(train_generator, 
            steps_per_epoch=10,
            epochs=1, 
            callbacks=[early_stop, checkpoint, rate_reduction],
            validation_data=valid_generator)

10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 750ms/step - accuracy: 0.7501 - loss: 0.5622 - val_accuracy: 0.5000 - val_loss: 0.8385 - learning_rate: 1.0000e-04


In [86]:
if not os.path.isdir('modelWeights/'):
    os.mkdir("modelWeights/")
model_02.save(filepath = "modelWeights/vgg19_model_02.keras", overwrite=True)

In [87]:
model_02.load_weights("modelWeights/vgg19_model_02.keras")

vgg_val_eval_02 = model_02.evaluate(valid_generator)
vgg_test_eval_02 = model_02.evaluate(test_generator)

print(f"Validation Loss: {vgg_val_eval_02[0]}")
print(f"Validation Accuarcy: {vgg_val_eval_02[1]}")
print(f"Test Loss: {vgg_test_eval_02[0]}")
print(f"Test Accuarcy: {vgg_test_eval_02[1]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.5000 - loss: 0.8385
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 550ms/step - accuracy: 0.5959 - loss: 0.7259
Validation Loss: 0.8385134935379028
Validation Accuarcy: 0.5
Test Loss: 0.684575080871582
Test Accuarcy: 0.625


In [ ]:
# Fine tuning the entire network (Second time)

base_model = VGG19(include_top=False, input_shape=(image_size, image_size,3))

x = base_model.output
flat = Flatten()(x)

class_1 = Dense(4608, activation = 'relu')(flat)
dropout = Dropout(0.2)(class_1)
class_2 = Dense(1152, activation = 'relu')(dropout)
output = Dense(2, activation = 'softmax')(class_2)

model_03 = Model(base_model.inputs, output)
model_03.load_weights("modelWeights/vgg19_model_02.keras")

print(model_03.summary())

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 4608)           │    37,753,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 4608)           │             

 Total params: 63,089,602 (240.67 MB)

 Trainable params: 63,089,602 (240.67 MB)

 Non-trainable params: 0 (0.00 B)

None


In [100]:
# Stochatic Gradient Descent
sgd = SGD(learning_rate= 0.0001, # Low initial value = more gradual updates
        momentum= 0.01, # TODO: play around with the momentum
        nesterov=True)
model_03.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])

In [101]:
history_03 = model_03.fit(train_generator, 
            steps_per_epoch=100,
            epochs=1, 
            callbacks=[early_stop, checkpoint, rate_reduction],
            validation_data=valid_generator)

ValueError: Input 0 of layer "functional_11" is incompatible with the layer: expected shape=(None, 128, 128, 3), found shape=(None, 150, 150, 3)

In [ ]:
if not os.path.isdir('modelWeights/'):
    os.mkdir("modelWeights/")
model_03.save(filepath = "modelWeights/vgg_unfrozen.keras", overwrite=True)